<a href="https://colab.research.google.com/github/looopic/ML2Project/blob/main/ML2Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Visual text recognition model by Carlo Huser (husercar@students.zhaw.ch)


This model is used to extract the ingredients from an image of groceries. I want to use this model to build an app to detect allergies or intolerances on specific ingredients.

# Data
I was thinking of making my own dataset for this problem, but I quickly found out, that it's too time consuming to annotate every single picture on my own.
Therefore I searched the internet for a dataset I could use and found the TextOCR dataset on kaggle (https://www.kaggle.com/datasets/robikscube/textocr-text-extraction-from-images-dataset?select=annot.csv)

In [6]:
import shutil
import os

shutil.rmtree('/content', ignore_errors=True)
os.makedirs('/content', exist_ok=True)

In [8]:
!gdown 1zg0ofaVhgnJQ4bmfLeo5MG1iAtoT2HnO
!unzip OCR_data.zip

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
Traceback (most recent call last):
  File "/usr/local/bin/gdown", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/gdown/cli.py", line 151, in main
    filename = download(
  File "/usr/local/lib/python3.10/dist-packages/gdown/download.py", line 247, in download
    f = open(tmp_file, "ab")
FileNotFoundError: [Errno 2] No such file or directory: 'OCR_data.zip334ck2q1tmp'
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
unzip:  cannot find or open OCR_data.zip, OCR_data.zip.zip or OCR_data.zip.ZIP.


# Preprocess Images

Due to the images being real photographs, there are multiple words on each image. I need to extract those word on their own, so that i can label that word.

In [2]:
import csv
import cv2
import numpy as np
import math
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

In [3]:
def roundup(x):
    return int(math.ceil(x / 10.0)) * 10

In [4]:
def preprocess(img):
	old_h,old_w=img.shape[0],img.shape[1]
  
	#Pad the height.
	#If height is less than 512 then pad to 512
	if old_h<512:
		to_pad=np.ones((512-old_h,old_w,3))*255
		img=np.concatenate((img,to_pad))
		new_height=512
	else:
	#If height >512 then pad to nearest 10.
		to_pad=np.ones((roundup(old_h)-old_h,old_w,3))*255
		img=np.concatenate((img,to_pad))
		new_height=roundup(old_h)

	#Pad the width.
	if old_w<512:
		to_pad=np.ones((new_height,512-old_w,3))*255
		img=np.concatenate((img,to_pad),axis=1)
		new_width=512
	else:
		to_pad=np.ones((new_height,roundup(old_w)-old_w,3))*255
		img=np.concatenate((img,to_pad),axis=1)
		new_width=roundup(old_w)-old_w
	return img

In [5]:
# initialize arrays to store data and labels
images = []
labels = []

# Create directories to store the images
image_dir = '/content/preprocessed_images/'
os.makedirs(image_dir, exist_ok=True)

# read annotations csv file
with open('/content/annot.csv','r') as csvfile:
  reader = csv.reader(csvfile)
  
  #skip header
  next(reader)

  for row in tqdm(reader):
    image_path = "/content/train_val_images/train_images/"+row[2]+".jpg"
    boundary = row[5].strip('[]').split(',')
    boundaryArray = [float(value) for value in boundary]
    boundaryXmin = int(min(boundaryArray[0], boundaryArray[2], boundaryArray[4], boundaryArray[6]))
    boundaryXmax = int(max(boundaryArray[0], boundaryArray[2], boundaryArray[4], boundaryArray[6]))
    boundaryYmin = int(min(boundaryArray[1], boundaryArray[3], boundaryArray[5], boundaryArray[7]))
    boundaryYmax = int(max(boundaryArray[1], boundaryArray[3], boundaryArray[5], boundaryArray[7]))
    text = row[4]

    #load image
    image = cv2.imread(image_path)

    #extract text region
    text_region = image[boundaryYmin:boundaryYmax, boundaryXmin:boundaryXmax]

    #preprocess text region
    text_region = preprocess(text_region)

    # Generate a unique filename for each image
    image_filename = f"{row[2]+text}.jpg"

    # Save the preprocessed image to disk
    image_path = os.path.join(image_dir, image_filename)
    cv2.imwrite(image_path, text_region)

    #append data to arrays
    images.append(image_filename)
    labels.append(text)

#encode labels to numerical values
label_encoder = LabelEncoder()
label_encoder.fit(labels)
encoded_labels = label_encoder.transform(labels)


197it [00:07, 28.04it/s]


KeyboardInterrupt: ignored

# Model

I'm following the tutorial on medium.com (https://medium.com/geekculture/building-a-complete-ocr-engine-from-scratch-in-python-be1fd184753b)

# Evaluation and Comparision